# 数据加载、角度运算、初始化叠掩阴影、直方图匹配

In [1]:
import ee
import geemap
import geemap.chart as chart
import numpy as np
import pandas as pd
import copy
import math
import sys,os
sys.path.append(os.path.dirname(os.path.abspath('__file__')))
from tqdm import trange,tqdm
from Basic_tools import *
from New_Correct import *
from Correct_filter import *
from S2_filter import *
from functools import partial
import geopandas as gpd
geemap.set_proxy(port=10809)
# ee.Authenticate()
ee.Initialize()
print('geemap version = {}\ngeemap path = {}'.format(geemap.__version__,geemap.__path__))

D:\Wrz\anaconda\envs\GEE\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


geemap version = 0.20.0
geemap path = ['D:\\Wrz\\anaconda\\envs\\GEE\\lib\\site-packages\\geemap']


In [2]:
Glacial_lake_2015A = ee.FeatureCollection(
    'projects/ee-mrwurenzhe/assets/Glacial_lake/Wu_Asia_Southest_GL_wgs84').filter(ee.Filter.gte('GL_Area', 0.1))
    #projects/ee-mrwurenzhe/assets/Glacial_lake/Checkout_polygen

# 计算geometry、质心点、最小包络矩形
Geo_ext = lambda feature: feature.set({
    'Geo': feature.geometry(),
    'Centroid': feature.geometry().centroid(),
    'Rectangle': feature.geometry().bounds()
})

Centrid_set = lambda feature: feature.setGeometry(feature.geometry().centroid())
Rectangle_set = lambda feature: feature.setGeometry(feature.geometry().bounds())
Glacial_lake_2015C = Glacial_lake_2015A.map(Geo_ext).map(Centrid_set)  # 添加属性,修改geometry,计算质心
Glacial_lake_2015R = Glacial_lake_2015A.map(Rectangle_set)       # 计算最小包络矩形

#抽取属性作为list
Glacial_lake_2015A_GeoList = ee.List(Glacial_lake_2015C.reduceColumns(ee.Reducer.toList(), ['Geo']).get('list'))
Glacial_lake_2015C_CentriodList = ee.List(Glacial_lake_2015C.reduceColumns(ee.Reducer.toList(),['Centroid']).get('list'))
Glacial_lake_2015R_RectangleList = ee.List(Glacial_lake_2015C.reduceColumns(ee.Reducer.toList(),['Rectangle']).get('list'))
Num_list = Glacial_lake_2015C_CentriodList.size().getInfo()

In [3]:
# 选择一个时间点的影像，注意SAR和光学影像的时间点已经不一样了
# 需要放宽一些时间限制，否则可能会因为天气影响、卫星维护等问题出现数据缺失
START_DATE  = ee.Date('2019-07-01')
END_DATE   = ee.Date('2019-08-30')
TIME_LEN   = END_DATE.difference(START_DATE, 'days').abs()
MIDDLE_DATE  = START_DATE.advance(TIME_LEN.divide(ee.Number(2)).int(),'days')

DEMSRTM = ee.Image('USGS/SRTMGL1_003')
DEM_prj = DEMSRTM.projection()
DEMNASA = ee.Image("NASA/NASADEM_HGT/001").select('elevation')
DEMALOS = ee.ImageCollection("JAXA/ALOS/AW3D30/V3_2").mosaic().select('DSM').rename('elevation').reproject(DEM_prj)
DEMCOPERNICUS = ee.ImageCollection("COPERNICUS/DEM/GLO30").mosaic().select('DEM').rename('elevation').int16().reproject(DEM_prj)


models = ['volume', 'surface', None]     # 地形矫正模型
Model = models[0]
Origin_scale = 10
projScale = 30


# 选择一个点做实验
i=1143 #394(面积大) #53 #11(影像都用nodata) #149(坐标系问题) #1143(畸变) #539(中等交并比)  #1296(高等交并比)

AOI_point = ee.Feature.geometry(Glacial_lake_2015C_CentriodList.get(i))
AOI_area = ee.Feature.geometry(Glacial_lake_2015A_GeoList.get(i))
AOI = ee.Feature.geometry(Glacial_lake_2015R_RectangleList.get(i))

# 计算面积
AOI_area_area = AOI_area.area().divide(ee.Number(1000*1000)).getInfo()
# 缩小
if AOI_area_area < 1:
    AOI_area_buffer = AOI_area.buffer(distance=AOI_area_area*-300)
else:
    AOI_area_buffer = AOI_area.buffer(distance=-400)

# 扩大包络矩形AOI,保证背景像素占比最大，原有基础上相应+100
if AOI_area_area < 1:
    AOI_buffer = AOI.buffer(distance=400).bounds()
else:
    AOI_buffer = AOI.buffer(distance=500).bounds()

# 载入图像，采用滤波函数，筛选日期，AOI_buffer仅用于统计是否有空值点
s1_ascending, s1_descending = load_image_collection(AOI_buffer,START_DATE,END_DATE,MIDDLE_DATE,
                                                            Filter='RefinedLee',FilterSize=30)

CLOUD_FILTER = 60           # 过滤s2 大于指定云量的数据
CLD_PRB_THRESH = 15         # s2cloudless 概率值阈值[0-100],原实验是50
NIR_DRK_THRESH = 0.15       # 非水暗像素判断阈值
CLD_PRJ_DIST = 1            # 根据 CLD_PRJ_DIST 输入指定的距离从云中投射阴影
BUFFER = 50                 # Remove small cloud-shadow patches and dilate remaining pixels by BUFFER input
# 采用S2cloudless生产一张合成的无云图像
s2_sr_median = merge_s2_collection(AOI_buffer,START_DATE,END_DATE,
                      CLOUD_FILTER,BUFFER,CLD_PRJ_DIST,CLD_PRB_THRESH,NIR_DRK_THRESH)

Begin Filter ...


# 计算几何畸变区域，然后采用ImageCollection进行融合

In [4]:
def CalDitorAndCombin(s1_ascending,s1_descending,Bands_ = ['VV_gamma0_flatDB','VH_gamma0_flatDB'],):
  # 当'synthesis': 1，则跳过几何畸变校正步骤，直接进入识别
  synthesis_a  = ee.Image(s1_ascending).get('synthesis').getInfo()
  synthesis_d  = ee.Image(s1_descending).get('synthesis').getInfo()
  if synthesis_a or synthesis_d:
      print('Image with synthesis')
      Combin_ad = ee.ImageCollection([s1_ascending,s1_descending])
      s1_unit_mean_ = Combin_ad.mean()
      s1_unit_max_  = Combin_ad.max()
      s1_unit_min_  = Combin_ad.min()
  else:
      print('Image with slop correction and volumetric')
      volumetric_dict = my_slope_correction(s1_ascending,s1_descending,AOI_buffer,DEMNASA,Model,Origin_scale)

  # ----------------------Z-scor均衡化，直方图匹配，在局部进行直方图匹配
  Bands_ = Bands_
  Ascending_Img = volumetric_dict['ASCENDING'].select(Bands_)
  Descending_Img = volumetric_dict['DESCENDING'].select(Bands_)

  ASCENDING_meanStd = meanStd_norm(Ascending_Img,Bands_,scale=Origin_scale)
  DESSCENDING_meanStd = meanStd_norm(Descending_Img,Bands_,scale=Origin_scale)

  # 经过z-score，无需再进行均值迁移
  Match_Ascending = histogramMatching(ASCENDING_meanStd, DESSCENDING_meanStd
                      ,AOI_buffer,Bands_,Bands_
                      ,Histscale=projScale,maxBuckets=1024).clip(AOI_buffer)

  volumetric_dict['ASCENDING'] = replaceBands(volumetric_dict['ASCENDING'],Match_Ascending)
  volumetric_dict['DESCENDING'] = replaceBands(volumetric_dict['DESCENDING'],DESSCENDING_meanStd)


  # -----------------------基于线性关系。检测几何畸变
  Templist_A = AuxiliaryLine2Point(volumetric_dict['ASCENDING'],volumetric_dict['ASCENDING_parms']['s1_azimuth_across'],
                                volumetric_dict['ASCENDING_parms']['coordinates_dict'],
                                volumetric_dict['ASCENDING_parms']['Auxiliarylines'],
                                projScale)
  LeftLayoverA,RightLayoverA,ShadowA = Line_Correct(volumetric_dict['ASCENDING'],AOI_buffer,Templist_A,'ASCENDING',
                                                volumetric_dict['ASCENDING_parms']['proj'],projScale,Origin_scale)

  Templist_D = AuxiliaryLine2Point(volumetric_dict['DESCENDING'],volumetric_dict['DESCENDING_parms']['s1_azimuth_across'],
                                volumetric_dict['DESCENDING_parms']['coordinates_dict'],
                                volumetric_dict['DESCENDING_parms']['Auxiliarylines'],
                                projScale)
  LeftLayoverD,RightLayoverD,ShadowD =  Line_Correct(volumetric_dict['DESCENDING'],AOI_buffer,Templist_D,'DESCENDING',
                                                volumetric_dict['DESCENDING_parms']['proj'],projScale,Origin_scale)


  def Cal_mask(LeftLayover,RightLayover,Shadow,AOI_buffer):
      # 判断是否为空
      left_empty = LeftLayover.bandNames().length().eq(0)
      right_empty = RightLayover.bandNames().length().eq(0)
      shadow_empty = Shadow.bandNames().length().eq(0)

      # 只合并非空图像
      result = ee.Image(ee.Algorithms.If(left_empty, ee.Image(), LeftLayover))
      result = ee.Image(ee.Algorithms.If(right_empty,result,result.Or(RightLayover)))
      result = ee.Image(ee.Algorithms.If(shadow_empty,result,result.Or(Shadow)))
      return result.clip(AOI_buffer)

  LeftLayoverA,RightLayoverA,ShadowA = LeftLayoverA.mask(),RightLayoverA.mask(),ShadowA.mask()
  LeftLayoverD,RightLayoverD,ShadowD = LeftLayoverD.mask(),RightLayoverD.mask(),ShadowD.mask()

  A_mask_ = Cal_mask(LeftLayoverA,RightLayoverA,ShadowA,AOI_buffer)
  D_mask_ = Cal_mask(LeftLayoverD,RightLayoverD,ShadowD,AOI_buffer)
  All_distor = A_mask_.And(D_mask_)

  A_empty = A_mask_.bandNames().contains('constant')
  A_empty = ee.Number(ee.Algorithms.If(A_empty, 1, 0))
  D_empty = D_mask_.bandNames().contains('constant')
  D_empty = ee.Number(ee.Algorithms.If(D_empty, 1, 0))

  s1_ascending  = volumetric_dict['ASCENDING'].select(["VV_gamma0_flatDB","VH_gamma0_flatDB"])
  s1_descending = volumetric_dict['DESCENDING'].select(["VV_gamma0_flatDB","VH_gamma0_flatDB"])

  s1_ascending_ = ee.Image(ee.Algorithms.If(A_empty, ee.Image(), s1_ascending.where(A_mask_,s1_descending)))
  s1_descending_ = ee.Image(ee.Algorithms.If(D_empty, ee.Image(), s1_descending.where(D_mask_,s1_ascending)))
  Combin_AD = ee.ImageCollection([s1_ascending_,s1_descending_])

  s1_unit_mean_ = ee.Image(ee.Algorithms.If(A_empty.Or(D_empty),
                s1_ascending.add(s1_descending).divide(2),
                Combin_AD.mean()))

  s1_unit_max_ = ee.Image(ee.Algorithms.If(A_empty.Or(D_empty),
                s1_ascending.max(s1_descending),
                Combin_AD.max()))

  s1_unit_min_ = ee.Image(ee.Algorithms.If(A_empty.Or(D_empty),
                s1_ascending.min(s1_descending),
                Combin_AD.min()))
  return volumetric_dict,s1_unit_mean_,s1_unit_max_,s1_unit_min_

volumetric_dict,s1_unit_mean_,s1_unit_max_,s1_unit_min_ = \
            CalDitorAndCombin(s1_ascending,s1_descending,Bands_ = ['VV_gamma0_flatDB','VH_gamma0_flatDB'])

Image with slop correction and volumetric


100%|██████████████████████████████████████████████████████████████████████████████████| 45/45 [00:00<00:00, 57.24it/s]


In [5]:
Map = geemap.Map() # basemap='HYBRID'
Map.centerObject(AOI_buffer, zoom=15)
# Map.addLayer(volumetric_dict['ASCENDING'].select('red','green','blue'), {'min':0,'max':255}, 'no_data_maskrgb')
# Map.addLayer(LeftLayover.randomVisualizer(),{},'LeftLayover')
# Map.addLayer(RightLayover.randomVisualizer(),{},'RightLayover')
# Map.addLayer(Shadow.randomVisualizer(),{},'Shadow')
Map.addLayer(s2_sr_median,{'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 2000, 'gamma': 2.0},'s2_sr_median')
Map.addLayer(A_mask_.randomVisualizer(),{},'A_mask')
Map.addLayer(D_mask_.randomVisualizer(),{},'D_mask')
Map.addLayer(volumetric_dict['ASCENDING'].select('red','green','blue'), {'min':0,'max':255}, 'no_data_maskrgb_A')
Map.addLayer(volumetric_dict['DESCENDING'].select('red','green','blue'), {'min':0,'max':255}, 'no_data_maskrgb_D')
Map.addLayer(All_distor.randomVisualizer().clip(AOI_buffer), {}, 'All_distor')

Map.addLayer(volumetric_dict['ASCENDING'].select("VV_sigma0"),{'min':-18,'max':5},'ascending')
Map.addLayer(volumetric_dict['DESCENDING'].select("VV_sigma0"),{'min':-18,'max':5},'descending')

Map.addLayer(volumetric_dict['ASCENDING'].select("VV_gamma0_flatDB"),{'min':-2,'max':2},'ascending_flat')
Map.addLayer(volumetric_dict['DESCENDING'].select("VV_gamma0_flatDB"),{'min':-2,'max':2},'descending_flat')

Map.addLayer(s1_unit_mean_.select("VV_gamma0_flatDB"),{'min':-2,'max':2},'s1_unit_mean_')
Map.addLayer(s1_unit_max_.select("VV_gamma0_flatDB"),{'min':-2,'max':2},'s1_unit_max_')
Map.addLayer(s1_unit_min_.select("VV_gamma0_flatDB"),{'min':-2,'max':2},'s1_unit_min_')
Map

Map(center=[30.17633983666919, 94.28168536063386], controls=(WidgetControl(options=['position', 'transparent_b…

# 提取冰湖，检测效果

In [6]:
# MeanStd归一化，保留中间90%,以-2-2近似替代
def Percent90(Image:ee.Image):
    Image = Image.where(Image.gte(2),2)
    Image = Image.where(Image.lte(-2),-2)
    return Image

In [7]:
from Extract_algorithm import Cluster_extract as Cluster

def Cluster_math(method: str, img, bands: list, index: str, visual: bool, save: bool, region=None):
    '''method ('Kmean','SNIC','SNIC_Kmean','LVQ','Xmeans','Cobweb','CascadeKMeans')'''
    img = img.select(bands)
    img = Percent90(img).clip(region)
    img = minmax_norm(img,bands,scale=Origin_scale)

    if method == 'Kmean':
        result = Cluster.afn_Kmeans(img, region)
    elif method == 'Cobweb':
        result = Cluster.afn_Cobweb(img, region)
    elif method == 'Xmeans':
        result = Cluster.afn_Xmeans(img, region)
    elif method == 'LVQ':
        result = Cluster.afn_LVQ(img, region)
    elif method == 'CascadeKMeans':
        result = Cluster.afn_CascadeKMeans(img, region)
    elif method == 'SNIC':
        result = Cluster.afn_SNIC(img)
        result = result.select(result.bandNames().removeAll(['clusters', 'seeds']))
        result = result.reproject(result.select(0).projection().getInfo()['crs'], None, 10)
    elif method == 'SNIC_Kmean':
        result = Cluster.afn_SNIC(img)
        # 默认舍弃cluster和seed
        result = result.select(result.bandNames().removeAll(['clusters', 'seeds']))
        result = result.reproject(result.select(0).projection().getInfo()['crs'], None, 10)
        result0 = Cluster.afn_Kmeans(result, region)  # 原始图像不参与
        result1 = Cluster.afn_Kmeans(result.addBands(img), region)  # 原始图像参与    .unmask(10)

    if visual:
        Map = geemap.Map(basemap='HYBRID')  #
        Map.centerObject(AOI_point, zoom=15)
        Map.addLayer(img, {'min': 0.2, 'max': 0.8}, 'Origin')
        if method in ['Kmean', 'Cobweb', 'Xmeans', 'LVQ', 'CascadeKMeans']:
            Map.addLayer(result.randomVisualizer(), {}, method)
        elif method == 'SNIC':
            Map.addLayer(result.randomVisualizer(), {}, method)
        elif method == 'SNIC_Kmean':
            Map.addLayer(result0.randomVisualizer(), {}, 'SNIC_Kmean_NoOrigin')
            Map.addLayer(result1.randomVisualizer(), {}, 'SNIC_Kmean_YesOrigin')
        else:
            print('Please check your method str')
    else:
        Map = None
    if save:
        if method == 'SNIC_Kmean':
            Geemap_export(filename=index + 'NoOrigin' + method + '.tif', collection=False, image=result0,
                          region=region, scale=10)
            Geemap_export(filename=index + 'YesOrigin' + method + '.tif', collection=False, image=result1,
                          region=region, scale=10)
        else:
            Geemap_export(filename=index + method + '.tif', collection=False, image=result, region=region, scale=10)
        pass
    if method == 'SNIC_Kmean':
        return Map, result0.addBands(result1)
    else:
        return Map, result

Map,result=Cluster_math(method='Kmean',img=s1_unit_mean_,bands=['VV_gamma0_flatDB', 'VH_gamma0_flatDB'],
                        index='',visual=True,save=False,region=AOI_buffer)  # 如果迭代多个影像可以采用调整index
# Map

In [ ]:
from Extract_algorithm import Adaptive_threshold as Adap

def Bandmath(method:str,img,band,index:str,visual:bool,save:bool,region=None):
    '''
    method = ['otsu','histPeak']
    img: 仅单波段图像
    '''
    img = img.select(band)
#     assert len(img.bandNames().getInfo()) == 1 , '图像波段数必须唯一'
    img = Percent90(img).clip(region)
    img = minmax_norm(img,band,scale=Origin_scale)

    if method == 'otsu':
        histogram = get_histogram(img,region=region)
        Threshould_value = Adap.afn_otsu(histogram)
        result = img.select(0).gt(Threshould_value)  #
        print('Threshould value is {}'.format(Threshould_value.getInfo()))

    elif method == 'histPeak':
        Threshould_value = Adap.afn_histPeak(img,region=region)
        result = img.gt(Threshould_value)
        print('Threshould value is {}'.format(Threshould_value))

    if visual:
        Map = geemap.Map(basemap='HYBRID') #
        Map.centerObject(AOI_point, zoom=15)
        if method in ['otsu','histPeak']:
            Map.addLayer(result.randomVisualizer(), {}, method)
        else:
            print('Wrong visual! Please check your method str')

    if save:
        if method in ['otsu','histPeak']:
            Geemap_export(filename=index+method+'.tif',collection=False,
                          image=result,region=region,scale=10)
    else:
        print('Wrong save! Please check your method str')

    return Map,result

Map,result = Bandmath(method='histPeak',img=s1_unit_mean_,band=['VV_gamma0_flatDB'],index='',visual=True,save=False,region=AOI,)
Map

In [ ]:
method='histPeak';img=s1_unit_mean_;band=['VV_gamma0_flatDB'];index='';visual=True;save=False;region=AOI

In [ ]:
img = img.select(band)
assert len(img.bandNames().getInfo()) == 1 , '图像波段数必须唯一'
img = Percent90(img).clip(region)
img = minmax_norm(img,band,scale=Origin_scale)

In [ ]:
if method == 'otsu':
    histogram = get_histogram(img,region=region)
    Threshould_value = Adap.afn_otsu(histogram)
    result = img.select(0).gt(Threshould_value)  #
    print('Threshould value is {}'.format(Threshould_value.getInfo()))

elif method == 'histPeak':
    Threshould_value = Adap.afn_histPeak(img,region=region)
    result = img.gt(Threshould_value)
    print('Threshould value is {}'.format(Threshould_value))

# 输出图像

In [ ]:
from Extract_algorithm import Reprocess,save_parms
os.chdir('/content/drive/My Drive/Output/2018预测数据')
Methods = ('SNIC_Kmean',)
resultbands = (0,1)
Bands = ([0,1],)
mode='gpd'
logname = 'preSNIC_Kmean.csv'
shpname = 'preSNIC_Kmean.shp'

for i in trange(Num_list):
    AOI_point = ee.Feature.geometry(Glacial_lake_2015C_CentriodList.get(i))
    AOI_area = ee.Feature.geometry(Glacial_lake_2015A_GeoList.get(i))
    AOI = ee.Feature.geometry(Glacial_lake_2015R_RectangleList.get(i))

    # 计算面积
    AOI_area_area = AOI_area.area().divide(ee.Number(1000*1000)).getInfo()
    # 缩小
    if AOI_area_area < 1:
        AOI_area_buffer = AOI_area.buffer(distance=AOI_area_area*-300)
    else:
        AOI_area_buffer = AOI_area.buffer(distance=-400)

    # 扩大包络矩形AOI,保证背景像素占比最大
    if AOI_area_area < 1:
        AOI_buffer = AOI.buffer(distance=300)
    else:
        AOI_buffer = AOI.buffer(distance=400)

    # 在这个点采用buffer裁剪文件，展示SAR，去除了包含空值的影像
    s1_ascending,s1_descending = load_image_collection(AOI_buffer,START_DATE,END_DATE,MIDDLE_DATE,
                            dem=DEMNASA,model=models[2],Filter='RefinedLee',FilterSize=30)

    # 全包含空值条件下，将图像集合合并为一张图像进行处理
    if type(s1_ascending) == ee.imagecollection.ImageCollection:
        s1_single_a = s1_ascending.filter(ee.Filter.eq('time_difference',
            s1_ascending.aggregate_min('time_difference'))).first().set({'synthesis': 0})
    else:
        s1_single_a = s1_ascending.set({'synthesis': 1})
    if type(s1_descending) == ee.imagecollection.ImageCollection:
        s1_single_d = s1_descending.filter(ee.Filter.eq('time_difference',
            s1_descending.aggregate_min('time_difference'))).first().set({'synthesis': 0})
    else:
        s1_single_d = s1_descending.set({'synthesis': 1})

    # 加入条件空值，保证在空值情况下图像能够正常相加
    condition = s1_single_d.mask().clip(AOI_buffer)
    s1_unit_mean = s1_single_a.where(condition, s1_single_a.add(s1_single_d).divide(2)) #转为均值
    s1_unit_max = s1_single_a.where(condition, s1_single_a.max(s1_single_d))
    # s1_unit_add = s1_single_a.where(condition, s1_single_a.add(s1_single_d))

    s1_unit_all = s1_unit_mean.addBands(s1_unit_max)

    # 采用S2cloudless生产一张合成的无云图像
    s2_sr_median = merge_s2_collection(AOI_buffer,ee.Date('2019-06-01'),ee.Date('2019-09-30'))

    for Method in Methods:
        if Method == 'SNIC_Kmean':
            K = 2
        else:
            K = 1
    for k in range(K):
        resultband = resultbands[k]
        for Band in Bands:
            Map,result=Cluster_math(method=Method,img=s1_unit_all,bands=Band,index=''
                    ,visual=False,save=False,region=AOI_buffer)

            if AOI_area_buffer.coordinates().getInfo() == []:
                FilterBound = AOI_area
            else:
                FilterBound = AOI_area_buffer

            # 分类图转矢量
            Union_ex = Reprocess.image2vector(result,
                        resultband=resultband,
                        GLarea=AOI_area_area,
                        FilterBound=FilterBound)
            if s1_single_d.get('synthesis').getInfo() == 0:
                d_name = s1_single_d.get('system:index').getInfo()
                d_date = s1_single_d.date().format('YYYY-MM-dd').getInfo()
                d_nodata = s1_single_d.get('numNodata').getInfo()
            else:
                d_name = 'None'; d_date = 'None' ; d_nodata='None'

            if s1_single_a.get('synthesis').getInfo() == 0:
                a_name = s1_single_a.get('system:index').getInfo()
                a_date = s1_single_a.date().format('YYYY-MM-dd').getInfo()
                a_nodata = s1_single_a.get('numNodata').getInfo()
            else:
                a_name = 'None'; a_date = 'None' ; a_nodata='None'

            pd_dict = {'a_name':a_name,'d_name':d_name,
                      'a_date':a_date,'d_date':d_date,
                      'a_nodata':a_nodata,'d_nodata':d_nodata}

            # 导出csv和shp
            save_parms.write_pd(Union_ex,i,mode=mode,Method=Method,Band=Band,WithOrigin=resultband,pd_dict=pd_dict,
                  Area_real=AOI_area_area,logname=logname,shapname=shpname,calIoU=True)

            # 加入升降轨影像名称，加入升降轨影像成像日期，加入升降轨道空值像素个数
            if k == 0:
                # 导出图像
                try:
                    Geemap_export(filename=f'{i:04d}'+'_'+str(resultband)+'Ascending_'+a_name+'.tif',collection=False,image=s1_single_a,region=AOI_buffer,scale=10)
                    Geemap_export(filename=f'{i:04d}'+'_'+str(resultband)+'Descending_'+d_name+'.tif',collection=False,image=s1_single_d,region=AOI_buffer,scale=10)
                    Geemap_export(filename=f'{i:04d}'+'_'+str(resultband)+'s2a_sr_median'+'.tif',collection=False,image=s2_sr_median,region=AOI_buffer,scale=10)
                except:
                    dir_name = make_dir(f'{i:04d}')
                    # 循环输出像素量超标的图像，(裁剪、挨个导出)
                    block_list = cut_geometry(AOI_buffer)
                    for each in range(len(block_list)):
                        Geemap_export(filename=os.path.join(dir_name,f'{i:04d}'+'_'+str(resultband)+'Ascending_'+a_name+'.tif')
                                    ,collection=False,image=s1_single_a,region=block_list[each],scale=10)
                        Geemap_export(filename=os.path.join(dir_name,f'{i:04d}'+'_'+str(resultband)+'Descending_'+d_name+'.tif')
                                    ,collection=False,image=s1_single_d,region=block_list[each],scale=10)
                        Geemap_export(filename=os.path.join(dir_name,f'{i:04d}'+'_'+str(resultband)+'s2a_sr_median'+'.tif')
                                    ,collection=False,image=s2_sr_median,region=block_list[each],scale=10)

## 不同的融合方法对应效果

In [ ]:
testCollection = ee.ImageCollection([volumetric_dict['ASCENDING'][2],volumetric_dict['DESCENDING'][2]])

testImage_mean = testCollection.mean()
testImage_std = testCollection.reduce(ee.Reducer.stdDev())
testImage_max = testCollection.max()
testImage_min = testCollection.min()

Map = geemap.Map()
Map.centerObject(AOI, zoom=8)
Map.addLayer(volumetric_dict['ASCENDING'][2].select('VV_gamma0flat'),{'min':-18,'max':5},'ASCENDING')
Map.addLayer(volumetric_dict['DESCENDING'][2].select('VV_gamma0flat'),{'min':-18,'max':5},'DESCENDING')
Map.addLayer(testImage_mean.select('VV_gamma0flat'),{'min':-18,'max':5},'mean')
Map.addLayer(testImage_max.select('VV_gamma0flat'),{'min':-18,'max':5},'max')
Map.addLayer(testImage_std.select('VV_gamma0flat_stdDev'),{'min':-18,'max':5},'std')
Map.addLayer(testImage_min.select('VV_gamma0flat'),{'min':-18,'max':5},'min')
Map

## 数据分布情况，并进行归一化与Z-norm

In [ ]:
import geemap.chart as chart
Bands = ["VV_gamma0_flatDB"]
ASCENDING_VV_gamma0flat = volumetric_dict['ASCENDING'].select(Bands)
DESCENDING_VV_gamma0flat = volumetric_dict['DESCENDING'].select(Bands)

# MinMax标准化
ASCENDING_Minmax = minmax_norm(ASCENDING_VV_gamma0flat,Bands,scale=10)
DESCENDING_Minmax = minmax_norm(DESCENDING_VV_gamma0flat,Bands,scale=10)
ASCENDING_MeanStd = meanStd_norm(ASCENDING_VV_gamma0flat,Bands,scale=10)
DESCENDING_MeanStd = meanStd_norm(DESCENDING_VV_gamma0flat,Bands,scale=10)

Match_Ascending = histogramMatching(ASCENDING_VV_gamma0flat, DESCENDING_VV_gamma0flat,AOI_buffer,Bands,Bands,Histscale=30,maxBuckets=1024)

In [ ]:
options = {
    "title": 'Asending Data Distribution',
    "xlabel": 'Pixel Value (mm)',
    "ylabel": 'Pixel Count',
    "colors": ['#1d6b99'],
}
my_sample = ASCENDING_VV_gamma0flat.sample(AOI_buffer, numPixels=10000,geometries=True)
property = 'VV_gamma0flat'
chart.feature_histogram(my_sample, property, **options)

In [ ]:
options = {
    "title": 'Descending Data Distribution',
    "xlabel": 'Pixel Value (mm)',
    "ylabel": 'Pixel Count',
    "colors": ['#1d6b99'],
}
my_sample = DESCENDING_VV_gamma0flat.sample(AOI_buffer, numPixels=10000,geometries=True)
property = 'VV_gamma0flat'
chart.feature_histogram(my_sample, property, **options)

In [ ]:
options = {
    "title": 'Ascending MeanStd Data Distribution',
    "xlabel": 'Pixel Value (mm)',
    "ylabel": 'Pixel Count',
    "colors": ['#1d6b99'],
}
my_sample = ASCENDING_MeanStd.sample(AOI_buffer, numPixels=10000,geometries=True)
property = 'VV_gamma0flat'
chart.feature_histogram(my_sample, property, **options)

In [ ]:
options = {
    "title": 'Descending MeanStd Data Distribution',
    "xlabel": 'Pixel Value (mm)',
    "ylabel": 'Pixel Count',
    "colors": ['#1d6b99'],
}
my_sample = DESCENDING_MeanStd.sample(AOI_buffer, numPixels=10000,geometries=True)
property = 'VV_gamma0_flatDB'
chart.feature_histogram(my_sample, property, **options)

In [ ]:
# Match_Asc
options = {
    "title": 'Ascending HistMatch Data Distribution',
    "xlabel": 'Pixel Value (mm)',
    "ylabel": 'Pixel Count',
    "colors": ['#1d6b99'],
}
my_sample = Match_Ascending.sample(AOI_buffer, numPixels=10000,geometries=True)
property = 'VV_gamma0_flatDB'
chart.feature_histogram(my_sample, property, **options)